In [79]:
import pandas as pd
import numpy as np
import censusdata
import re

In [19]:
# received in email by filling out https://api.census.gov/data/key_signup.html
# took a while after 'activating' to actually become active
KEY = "569913f1c2bf1c28df9b1f0c05f120b8691ef91b"

In [159]:
# pennsylvania is 42
state_dict = censusdata.geographies(censusdata.censusgeo([('state', '*'),]), 'acs5', 2015, key=KEY)

In [161]:
state_dict_lookup = {v.geo[0][1]: k for k, v in state_dict.items()}
state_dict_lookup

{'01': 'Alabama',
 '02': 'Alaska',
 '04': 'Arizona',
 '05': 'Arkansas',
 '06': 'California',
 '08': 'Colorado',
 '09': 'Connecticut',
 '10': 'Delaware',
 '11': 'District of Columbia',
 '12': 'Florida',
 '13': 'Georgia',
 '15': 'Hawaii',
 '16': 'Idaho',
 '17': 'Illinois',
 '18': 'Indiana',
 '19': 'Iowa',
 '20': 'Kansas',
 '21': 'Kentucky',
 '22': 'Louisiana',
 '23': 'Maine',
 '24': 'Maryland',
 '25': 'Massachusetts',
 '26': 'Michigan',
 '27': 'Minnesota',
 '28': 'Mississippi',
 '29': 'Missouri',
 '30': 'Montana',
 '31': 'Nebraska',
 '32': 'Nevada',
 '33': 'New Hampshire',
 '34': 'New Jersey',
 '35': 'New Mexico',
 '36': 'New York',
 '37': 'North Carolina',
 '38': 'North Dakota',
 '39': 'Ohio',
 '40': 'Oklahoma',
 '41': 'Oregon',
 '42': 'Pennsylvania',
 '44': 'Rhode Island',
 '45': 'South Carolina',
 '46': 'South Dakota',
 '47': 'Tennessee',
 '48': 'Texas',
 '49': 'Utah',
 '50': 'Vermont',
 '51': 'Virginia',
 '53': 'Washington',
 '54': 'West Virginia',
 '55': 'Wisconsin',
 '56': 'Wyoming

In [163]:
# allegheny is '003' for testing
county_dict = censusdata.geographies(censusdata.censusgeo([('state', '42'), ('county', '*')]), 'acs5', 2015, key=KEY)
sex_by_age_table_info_dict = censusdata.censustable('acs5', 2015, 'B01001')

In [171]:
county_dict_lookup = {v.geo[1][1]: " ".join(k.split(',')[0].split(' ')[:-1]) for k, v in county_dict.items()}
county_dict_lookup

{'001': 'Adams',
 '003': 'Allegheny',
 '005': 'Armstrong',
 '007': 'Beaver',
 '009': 'Bedford',
 '011': 'Berks',
 '013': 'Blair',
 '015': 'Bradford',
 '017': 'Bucks',
 '019': 'Butler',
 '021': 'Cambria',
 '023': 'Cameron',
 '025': 'Carbon',
 '027': 'Centre',
 '029': 'Chester',
 '031': 'Clarion',
 '033': 'Clearfield',
 '035': 'Clinton',
 '037': 'Columbia',
 '039': 'Crawford',
 '041': 'Cumberland',
 '043': 'Dauphin',
 '045': 'Delaware',
 '047': 'Elk',
 '049': 'Erie',
 '051': 'Fayette',
 '053': 'Forest',
 '055': 'Franklin',
 '057': 'Fulton',
 '059': 'Greene',
 '061': 'Huntingdon',
 '063': 'Indiana',
 '065': 'Jefferson',
 '067': 'Juniata',
 '069': 'Lackawanna',
 '071': 'Lancaster',
 '073': 'Lawrence',
 '075': 'Lebanon',
 '077': 'Lehigh',
 '079': 'Luzerne',
 '081': 'Lycoming',
 '083': 'McKean',
 '085': 'Mercer',
 '087': 'Mifflin',
 '089': 'Monroe',
 '091': 'Montgomery',
 '093': 'Montour',
 '095': 'Northampton',
 '097': 'Northumberland',
 '099': 'Perry',
 '101': 'Philadelphia',
 '103': 'Pike

In [172]:
# pennsylvania is 42

# get all county codes from above dictionary and loop through them with this
pa_state_code = '42'
all_age_ranges = ['B01001_{:03d}E'.format(i) for i in range(1,50)]
df_list = []
for county_code in county_dict_lookup:
    dlDF = censusdata.download('acs5', 
                           2018,
                           censusdata.censusgeo([('state', '42'), ('county', county_code), ('block group' , '*')]),
                           all_age_ranges,
                           key=KEY,
                          endpt='')

    new_cols = [re.sub("!!", "", re.sub(":", " ", sex_by_age_dict[col]['label'])) for col in all_age_ranges]

    sexageDF = dlDF.rename(columns = dict(zip(all_age_ranges, new_cols)))

    # https://www.census.gov/library/visualizations/2017/comm/voting-rates-age.html
    # combine 18-29, 30-44, 45-64, 65+ as age blocs of interest
    # print(sexageDF.columns)
    sexageDF['Male <18'] = sexageDF['Male Under 5 years'] + sexageDF['Male 5 to 9 years'] + \
                               sexageDF['Male 10 to 14 years'] + sexageDF['Male 15 to 17 years']
    sexageDF['Male 18-29'] = sexageDF['Male 18 and 19 years'] + sexageDF['Male 20 years'] + \
                                sexageDF['Male 21 years'] + sexageDF['Male 22 to 24 years']
    sexageDF['Male 30-44'] = sexageDF['Male 30 to 34 years'] + sexageDF['Male 35 to 39 years'] + \
                                sexageDF['Male 40 to 44 years']
    sexageDF['Male 45-64'] = sexageDF['Male 45 to 49 years'] + sexageDF['Male 50 to 54 years'] + \
                                sexageDF['Male 55 to 59 years'] + sexageDF['Male 60 and 61 years'] + \
                                sexageDF['Male 62 to 64 years']
    sexageDF['Male >=65'] = sexageDF['Male 65 and 66 years'] + sexageDF['Male 67 to 69 years'] + \
                                sexageDF['Male 70 to 74 years'] + sexageDF['Male 75 to 79 years'] + \
                                sexageDF['Male 80 to 84 years'] + sexageDF['Male 85 years and over']

    sexageDF['Female <18'] = sexageDF['Female Under 5 years'] + sexageDF['Female 5 to 9 years'] + \
                               sexageDF['Female 10 to 14 years'] + sexageDF['Female 15 to 17 years']
    sexageDF['Female 18-29'] = sexageDF['Female 18 and 19 years'] + sexageDF['Female 20 years'] + \
                                sexageDF['Male 21 years'] + sexageDF['Female 22 to 24 years']
    sexageDF['Female 30-44'] = sexageDF['Female 30 to 34 years'] + sexageDF['Female 35 to 39 years'] + \
                                sexageDF['Female 40 to 44 years']
    sexageDF['Female 45-64'] = sexageDF['Female 45 to 49 years'] + sexageDF['Female 50 to 54 years'] + \
                                sexageDF['Female 55 to 59 years'] + sexageDF['Female 60 and 61 years'] + \
                                sexageDF['Female 62 to 64 years']
    sexageDF['Female >=65'] = sexageDF['Female 65 and 66 years'] + sexageDF['Female 67 to 69 years'] + \
                                sexageDF['Female 70 to 74 years'] + sexageDF['Female 75 to 79 years'] + \
                                sexageDF['Female 80 to 84 years'] + sexageDF['Female 85 years and over']


    sexageDF[['Male <18', 'Female <18', 'Male 18-29', 'Female 18-29', 'Male 30-44', 'Female 30-44', 'Male 45-64', 'Female 45-64', 'Male >=65', 'Female >=65']]

    sexagelocDF = sexageDF.reset_index()
    sexagelocDF['state'] = sexagelocDF['index'].apply(lambda x: state_dict_lookup[x.geo[0][1]])
    sexagelocDF['county'] = sexagelocDF['index'].apply(lambda x: county_dict_lookup[x.geo[1][1]])
    sexagelocDF['tract'] = sexagelocDF['index'].apply(lambda x: x.geo[2][1])
    sexagelocDF['block_group'] = sexagelocDF['index'].apply(lambda x: x.geo[3][1])

    df_list.append(sexagelocDF[['state', 'county', 'tract', 'block_group', 'Male <18', 'Female <18', 'Male 18-29', 'Female 18-29', 'Male 30-44', 'Female 30-44', 'Male 45-64', 'Female 45-64', 'Male >=65', 'Female >=65']])


In [173]:
all_pa = pd.concat(df_list)

In [176]:
all_pa

,state,county,tract,block_group,Male <18,Female <18,Male 18-29,Female 18-29,Male 30-44,Female 30-44,Male 45-64,Female 45-64,Male >=65,Female >=65
0,Pennsylvania,Adams,030500,2,128,149,51,55,119,127,214,299,166,196
1,Pennsylvania,Adams,030500,1,237,214,136,102,186,154,355,331,195,161
2,Pennsylvania,Adams,031101,2,237,311,66,27,199,226,164,188,115,213
3,Pennsylvania,Adams,031101,3,164,141,114,64,163,94,317,371,202,262
4,Pennsylvania,Adams,031101,1,176,41,144,44,115,137,159,130,36,182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,Pennsylvania,York,023902,2,141,104,67,66,100,80,213,264,161,100
318,Pennsylvania,York,024001,3,62,75,62,16,31,45,178,133,43,50
319,Pennsylvania,York,024002,2,154,147,122,89,158,142,172,228,105,116
320,Pennsylvania,York,000200,3,80,32,51,0,55,12,58,75,11,10


In [175]:
all_pa.to_csv('age_sex_blockgrouplevel.csv', index=False, header=True)